# Estimation of overtopping failure probability (Hunt's equation)

In this example, we will demonstrate the application of various reliability methods to estimate the probability of failure of a levee due to overtopping. The failure mechanism is characterized by the Hunt's equation, which is a simple model to estimate the wave runup.

### Define model

First, we import the necessary package:

In [3]:
from ptk import *

We consider the following limit state function:

$Z = h_{crest} - (h + R_u)$

where: <br>

$h_{crest}$ is the crest of the flood defence [m] <br>
$h$ is the water level on the flood defence [m] <br>
$R_u$ represents the wave runup [m]

The wave runup $R_u$ is derived using the Hunt's equation:

$R_u = \xi \cdot H_s$

with:

$\xi = \frac{\tan (\alpha)}{\sqrt{2\cdot \pi \cdot H_s/L_0}} $

$L_0 = g \cdot T_p^2$

where:

$H_s$ is the significant wave height (m) <br>
$T_p$ is the peak wave period (s) <br>
$g$ is the acceleration of gravity ($9.81^2/s$) <br>
$\tan (\alpha)$ is the slope of the flood defence (-)

The parameters $h$, $H_s$ and $T_p$ represent the imposed load, while $h_{crest}$ and $\tan (\alpha)$ stand for the strength of the levee.

In [4]:
from ptk import *
import numpy as np

def hunt(T_p, tan_alpha, H_s, h_crest, h):
    g = 9.8
    L_0 = g  * T_p * T_p
    xi = tan_alpha / np.sqrt(2 * np.pi * H_s / L_0)
    Ru = xi * H_s

    z = h_crest - (h + Ru)
    return z

To perform a reliability analysis, we create a reliability project and specify the limit state function (model):

In [5]:
project = ReliabilityProject()
project.model = hunt

We assume the following distributions for the parameters present in the limit state function:

In [6]:
project.variables['T_p'].distribution = 'log_normal'
project.variables['T_p'].mean = 6
project.variables['T_p'].deviation = 2

project.variables['tan_alpha'].distribution = 'deterministic'
project.variables['tan_alpha'].mean = 0.333333

project.variables['H_s'].distribution = 'log_normal'
project.variables['H_s'].mean = 3
project.variables['H_s'].deviation = 1

project.variables['h_crest'].distribution = 'deterministic'
project.variables['h_crest'].mean = 10

project.variables['h'].distribution = 'exponential'
project.variables['h'].shift = 0.5
project.variables['h'].scale = 1

### Perform reliability calculations with Directional Sampling

We start with the reliability method `Directional Sampling`. We choose the calculation settings: `minimum_directions`, `maximum_directions` and `variation_coefficient`. The reliability analysis is executed using `project.run()`, and the results are accessed from `project.design_point`.

In [7]:
project.settings.reliability_method = "directional_sampling"

project.settings.minimum_directions = 10000
project.settings.maximum_directions = 20000
project.settings.variation_coefficient = 0.02

project.run()

def read_results(dp):

    beta = dp.reliability_index

    print(f"Beta = {beta}")

    pf = StandardNormal.get_q_from_u(beta)
    print(f"Probability of failure = {pf}")

    for alpha in dp.alphas:
        print(f"{alpha.variable.name}: alpha = {alpha.alpha}, x = {alpha.x}")

    if dp.is_converged:
        print(f"Converged (convergence = {dp.convergence} < {project.settings.variation_coefficient})")
    else:
        print(f"Not converged (convergence = {dp.convergence} > {project.settings.variation_coefficient})")

read_results(project.design_point)

Beta = 1.870995047897532
Probability of failure = 0.030672883709086787
T_p: alpha = -0.7670260126510028, x = 9.069371905507829
tan_alpha: alpha = 0.0, x = 0.333333
H_s: alpha = -0.3894230442845913, x = 3.6054016016523023
h_crest: alpha = 0.0, x = 10.0
h: alpha = -0.5099223357500876, x = 2.2718033942764047
Converged (convergence = 0.01965036823013643 < 0.02)


### Perform reliability calculations with Importance Sampling

We now conduct the reliability analysis using the `Importance Sampling`. We choose the calculation settings: `minimum_samples`, `maximum_samples` and `variation_coefficient`.

In [18]:
project.settings.reliability_method = "importance_sampling"

project.settings.minimum_samples = 1000
project.settings.maximum_samples = 30000
project.settings.variation_coefficient = 0.02

project.run()

read_results(project.design_point)

Beta = 1.8805459663276138
Probability of failure = 0.030016853957431904
T_p: alpha = -0.769003368904922, x = 9.101943087171737
tan_alpha: alpha = 0.0, x = 0.333333
H_s: alpha = -0.3787096751388942, x = 3.5862277100249296
h_crest: alpha = 0.0, x = 10.0
h: alpha = -0.5149881557561043, x = 2.293314969709919
Not converged (convergence = 0.030589007590459818 > 0.02)


### Perform reliability calculations with Adaptive Importance Sampling

We now conduct the reliability analysis using the `Adaptive Importance Sampling` method. We choose the calculation settings: `minimum_samples`, `maximum_samples`, `minimum_variance_loops`, `maximum_variance_loops`, `fraction_failed` and `variation_coefficient`.

In [19]:
project.settings.reliability_method = "adaptive_importance_sampling"

project.settings.minimum_samples = 1000
project.settings.maximum_samples = 30000
project.settings.minimum_variance_loops = 5
project.settings.maximum_variance_loops = 10
project.settings.fraction_failed = 0.5
project.settings.variation_coefficient = 0.02

project.run()

read_results(project.design_point)

Beta = 1.8824257802313953
Probability of failure = 0.029889114160469053
T_p: alpha = -0.7594004622192475, x = 9.052939902640759
tan_alpha: alpha = 0.0, x = 0.333333
H_s: alpha = -0.38454819316987027, x = 3.599876050199043
h_crest: alpha = 0.0, x = 10.0
h: alpha = -0.5248177065524574, x = 2.322672144572244
Converged (convergence = 0.019998732705313032 < 0.02)


### Perform reliability calculations with Subset Simulation

We now conduct the reliability analysis using the `Subset Simulation` method. We choose the calculation settings: `minimum_samples`, `maximum_samples` and `variation_coefficient`.

In [20]:
project.settings.reliability_method = "subset_simulation"

project.settings.minimum_samples = 1000
project.settings.maximum_samples = 30000
project.settings.variation_coefficient = 0.02

project.run()

read_results(project.design_point)

Beta = 1.7274752705362888
Probability of failure = 0.0420411713540157
T_p: alpha = -0.7668176613734434, x = 8.750001739111134
tan_alpha: alpha = 0.0, x = 0.333333
H_s: alpha = -0.393059150473663, x = 3.5478101749149626
h_crest: alpha = 0.0, x = 10.0
h: alpha = -0.5074398274029004, x = 2.1588658244514045
Converged (convergence = 0.01999290023051879 < 0.02)
